# Comparison of different PQCs #

In this notebook, we compare different PQCs inside Variational Quantum Algorithms. We select three PQC from Qiskit library: RealAmplitudes and EfficientSU2, which are displayed as follows:

In [2]:
from qiskit.circuit.library import RealAmplitudes, EfficientSU2


efficientSU2 = EfficientSU2(1, reps=1, entanglement='linear', insert_barriers=True)
realAmplitudes = RealAmplitudes(1,reps=1)

In [3]:
efficientSU2.draw(output='text')

┌───────────────────────────────────┐
q: ┤ EfficientSU2(θ[0],θ[1],θ[2],θ[3]) ├
   └───────────────────────────────────┘

In [4]:
realAmplitudes.draw(output='text')

┌───────────────────────────┐
q: ┤ RealAmplitudes(θ[0],θ[1]) ├
   └───────────────────────────┘

Now we compare execution times of Variational Quantum Eigensolver (VQE) by using both circuits. First, we do the necessary initializations. We initialize a Hermitian matrix to calculate eigenvalues:

In [6]:
import numpy as np

rdist = np.random.rand(1, 1)
rdistt = np.transpose(rdist)
z1 = np.zeros((1, 1))
z2 = np.zeros((1, 1))
matrix = np.block([[z1, rdist], [rdistt, z2]])

Then all parts needed for qiskit initialization:

In [7]:
from qiskit import IBMQ

from qiskit.providers.ibmq import least_busy
from qiskit.providers import JobStatus


IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q-research-2','vienna-uni-tech-1','main')

We select a target backend in the following way

In [10]:
backend = least_busy(
    provider.backends(
        filters=lambda x: x.configuration().n_qubits >= 3
        and not x.configuration().simulator
        and x.status().operational
        and x.configuration().dynamic_reprate_enabled
    )
)
print(backend.name())

ibmq_lima


First, we test execution on RealAmplitudes

In [16]:
from qiskit.algorithms.optimizers import COBYLA
from qiskit.opflow.primitive_ops import MatrixOp
import time

matrix_op = MatrixOp(matrix)

vqe_inputs = {
        'ansatz': realAmplitudes,
        'operator': matrix_op,
        'optimizer': COBYLA(max_iter=100, tol=0.00001),
        'initial_point': np.random.random(realAmplitudes.num_parameters),
        'measurement_error_mitigation': True,
        'shots': 1024
    }

options = {
        'backend_name': backend.name(),
    }

job = provider.runtime.run(program_id='vqe', inputs=vqe_inputs, options=options)
while job.status() != JobStatus.RUNNING:
    pass
start = time.time()
res = job.result()
end = time.time()
print("Runtime: "+str(end-start))

Runtime: 366.8015103340149


Now, we check time on EfficientSU2

In [18]:
from qiskit.algorithms.optimizers import COBYLA
from qiskit.opflow.primitive_ops import MatrixOp
import time

matrix_op = MatrixOp(matrix)

vqe_inputs = {
        'ansatz': efficientSU2,
        'operator': matrix_op,
        'optimizer': COBYLA(max_iter=100, tol=0.00001),
        'initial_point': np.random.random(efficientSU2.num_parameters),
        'measurement_error_mitigation': True,
        'shots': 1024
    }

options = {
        'backend_name': backend.name(),
    }

job = provider.runtime.run(program_id='vqe', inputs=vqe_inputs, options=options)
while job.status() != JobStatus.RUNNING:
    pass
start = time.time()
res = job.result()
end = time.time()
print("Runtime: "+str(end-start))

Runtime: 911.7872893810272


We notice a difference in ...

## Conclusion ##